In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
import numpy as np
private_labels=np.load("Nohot_private_label.npy")
private_pixels=np.load("private_pixels.npy")
public_labels=np.load("Nohot_public_label.npy")
public_pixels=np.load("public_pixels.npy")
train_labels=np.load("Nohot_train_label.npy")
train_pixels=np.load("train_pixels.npy")

In [3]:
X_train , y_train = train_pixels,train_labels
X_valid , y_valid = public_pixels,public_labels
X_test , y_test = private_pixels,private_labels

In [4]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train_centered = (X_train - mean_vals)/std_val
X_valid_centered = (X_valid - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val

In [5]:
import torch
import torch.nn
input_data=torch.FloatTensor(X_train_centered)
label=torch.LongTensor(y_train)

In [6]:
input_data1=torch.FloatTensor(X_valid_centered)
label1=torch.LongTensor(y_valid)

In [7]:
input_data2=torch.FloatTensor(X_test_centered)
label2=torch.LongTensor(y_test)

In [8]:
input_data=input_data.transpose(1,3)
input_data1=input_data1.transpose(1,3)
input_data2=input_data2.transpose(1,3)

In [9]:
input_data.shape,input_data1.shape,input_data2.shape

(torch.Size([28709, 1, 48, 48]),
 torch.Size([3589, 1, 48, 48]),
 torch.Size([3589, 1, 48, 48]))

In [10]:
label.shape,label1.shape,label2.shape

(torch.Size([28709]), torch.Size([3589]), torch.Size([3589]))

In [11]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
trainset = TensorDataset(input_data,label)
valset = TensorDataset(input_data1,label1)
testset = TensorDataset(input_data2,label2)

In [12]:
partition={'train':trainset,'val':valset,'test':testset}

In [16]:
class AlexNet_test(nn.Module):
    
    def __init__(self):
        super(AlexNet_test,self).__init__()
        self.feature=nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=4, stride=2, padding=2),# 64 26 26 => kernel remake
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2),# 64 13 13 => kernel remake
        nn.Conv2d(32, 128, kernel_size=4, padding=2), # 192, 13, 13 => kernel remake
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2), # 192 7 7 => kernel remake
        nn.Conv2d(128, 196, kernel_size=3, padding=1), # 384 7 7 => kernel remake
        nn.ReLU(inplace=True),
        nn.Conv2d(196, 384, kernel_size=3, padding=1), # 256 7 7
        nn.ReLU(inplace=True),
        nn.Conv2d(384, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2) # 256 6 6 
        )
        self.avgpool=nn.AdaptiveAvgPool2d((3,3)) # 256 6 6 (6,6)은 output size이다.
        # 평균을 구하는 것으로 반올림 정수가 나옴.
        self.classifier = nn.Sequential(
        nn.Dropout(),
        nn.Linear(256*3*3,1024),  # 9216=>4096, 256 36
        nn.ReLU(inplace=True),
        nn.Dropout(),
        nn.Linear(1024,1024),
        nn.ReLU(inplace=True),
        nn.Linear(1024,7),
        
        )
    def forward(self, x):
        x=self.feature(x)
        x=self.avgpool(x)
        x=torch.flatten(x,1) # ex => 2,2,2 => 2,4
        x=self.classifier(x)
        return x

In [17]:
def dimension_check():
    net=AlexNet_test()
    x=torch.randn(1,1,48,48)
    y=net(x)
    print(y.size())

In [18]:
dimension_check()

torch.Size([1, 7])


In [19]:
def train(net,partition,optimizer,criterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=64,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, 0)):
        time.sleep(0.0000001)
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [20]:
def test(net, partition):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=64, 
                                             shuffle=False, num_workers=2)
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [21]:
def validate(net, partition, criterion):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=64, 
                                            shuffle=False, num_workers=2)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in tqdm(valloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [22]:
from tqdm import tqdm, tqdm_notebook
from tqdm import trange
import time

In [23]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
net=AlexNet_test()
net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0015, weight_decay=0.00001)
train_losses=[]
val_losses=[]
train_accs=[]
val_accs=[]
for epoch in range(100):
    ts=time.time()
    net, train_loss,train_acc=train(net, partition, optimizer, criterion)
    val_loss, val_acc = validate(net, partition, criterion)
    te=time.time()
    print('time : {:2.2f}, train_loss : {:2.2f}, train_acc : {:2.2f},val_acc : {:2.2f},val_loss : {:2.2f}'.format(te-ts,train_loss,train_acc,val_acc,val_loss))
    train_loss=round(train_loss,2)
    val_loss=round(val_loss,2)
    train_acc=round(train_acc,2)
    val_acc=round(val_acc,2)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
test_acc = test(net,partition)
test_acc=round(test_acc,2)

  4%|▎         | 16/449 [00:00<00:02, 157.71it/s]

time : 3.14, train_loss : 1.80, train_acc : 24.88,val_acc : 26.50,val_loss : 1.77


  4%|▎         | 16/449 [00:00<00:02, 157.35it/s]

time : 2.85, train_loss : 1.73, train_acc : 29.50,val_acc : 35.89,val_loss : 1.58


  4%|▍         | 17/449 [00:00<00:02, 164.79it/s]

...

  4%|▎         | 16/449 [00:00<00:02, 153.84it/s]

time : 2.85, train_loss : 0.19, train_acc : 94.94,val_acc : 48.26,val_loss : 2.50


  0%|          | 0/57 [00:00<?, ?it/s]

time : 2.86, train_loss : 0.18, train_acc : 95.25,val_acc : 48.06,val_loss : 2.70


100%|██████████| 57/57 [00:00<00:00, 324.93it/s]


In [24]:
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/Alex_train_loss.npy",train_loss)
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/Alex_val_loss.npy",val_loss)
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/Alex_train_acc.npy",train_acc)
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/Alex_val_acc.npy",val_acc)